In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from numba import njit, prange
from source.cs import mh_sampler, draw_flat_prior, estimate_optimal_W_inv, L, ρ

In [2]:
# # @njit
# def adaptive_smc(J, K, B, α_star, bounds, args, seed=1):
#     """
#     Implements adaptive sequential Monte Carlo Algorthim.
#     Assume prior is flat.

#     Parameters
#     ----------
#     J : int
#         Number of iterations.
#     K : int
#         Length of MCMC chains.
#     B : int
#         Number of samples we want to draw.
#     α_star : tuple
#         Estimated optimal parameters.        
#     bounds : tuple of tuple of floats
#         Bounds on parameters.
#     args : tuple
#         Model parameters.

#     Returns
#     -------
#     res : (T, num_var) ndarrays
#         Sampled parameters.

#     """
#     ϕ = np.linspace(0, 1, J)
#     wj = np.ones(B)
#     αj = draw_flat_prior(bounds, B, seed)
#     W_inv = estimate_optimal_W_inv(α_star, args)
#     n = args[1].shape[0]
#     for j in prange(1, J):
#         print(j)
#         # Step 1: Correction
#         Lj = np.zeros(B)
#         for b in prange(B):
#             Lj[b] = L(αj[b], W_inv, args)
#         print(Lj)
#         vj = np.exp((ϕ[j]-ϕ[j-1])*n*Lj)
# #         print(vj)
#         wj = vj*wj
# #         print(wj)
#         wj = wj/np.sum(wj)
#         # Step 2: Selection
#         ESSj = B/np.mean(wj**2)
#         if ESSj > B/2.:
#             ellj = αj.copy()
#         else:
#             ellj = np.zeros(B)
#             for b in prange(B):
# #                 print(wj)
#                 idx = np.random.multinomial(1, wj).argmax()
#                 ellj[b] = αj[idx]
#             wj = np.ones(B)
#         # Step 3: Mutation
#         for b in prange(B):
#             temp = mh_sampler(ϕ[j], ellj[b], W_inv, args, step=0.1, T=K, seed=seed)
#             αj[b] = temp[-1, :]
#     return αj

# res = adaptive_smc(J=1000, K=100, B=10, α_star=α, bounds=tuple([[-0.1, 0.1]]*len(α)), args=(ξ, f, g, z1, z0, z0_float), seed=1)

In [3]:
# Load data
data = pd.read_csv('data/UnitaryData.csv')
pd_lag = np.array(data['d.p'])
n_states = 3

# Calculate indicator based on today's pd ratio
z0_float = np.empty((data.shape[0], n_states))
tercile = np.quantile(pd_lag, np.arange(n_states + 1)/n_states)
for i in range(n_states):
    z0_float[:,i] = (pd_lag >= tercile[i]) & (pd_lag <= tercile[i+1])
z0 = z0_float.astype(bool)

# Calculate indicator for tomorrow's pd ratio
z1 = z0[1:]

# Drop last row since we do not have tomorrow's pd ratio at that point
z0 = z0[:-1]
z0_float = z0_float[:-1]
f = np.array(data[['Rf','Rm-Rf','SMB','HML']])[:-1]
g = np.array(data['log.RW'])[:-1]
ξ = 0.14226531982421875 # 20% higher than min, lower bound problem

args = (ξ, f, g, z1, z0, z0_float)

In [4]:
μ = 0.00919840162386765
λ = np.array([-1.68, 3.68, -0.39, -0.87, -1.36, 3.36, 0.73, -6.54, -0.28, 2.28, -3.77, -7.08])
v = np.array([0.14115504, 0.29022421]) # v[i]-v[0], i=1,2
α = np.hstack([μ, λ, v])

In [6]:
W_inv = estimate_optimal_W_inv(α, args)

In [7]:
res, rate = mh_sampler(1., α, W_inv, args, step=5e-3, T=10_000, seed=1)

In [8]:
rate

0.4436

In [34]:
L(α, W_inv, args)

-1.8474439174306716e-07

In [17]:
α.shape

(15,)

In [46]:
def PL_objective_minus(η, μ, W_inv, args):
    α = np.hstack((μ, η))
    res = L(α, W_inv, args)
    return -res


def PL(μ, W_inv, args, tol, max_iter):
    ξ, f, g, z1, z0, z0_float = args
    n_f = f.shape[1]
    n_states = z1.shape[1]
    for method in ['Nelder-Mead']:
        model = minimize(PL_objective_minus,
                         np.zeros((n_f+1)*n_states-1),
                         args=(μ, W_inv, args),
                         method=method,
                         tol=tol,
                         options={'maxiter': max_iter, 'disp':True})
        if model.success:
            break
    if model.success == False:
        print("---Warning: the convex solver fails when tolerance = %s--- " % (tol))
        print(model.message)
    return -model.fun, model.x

In [47]:
y, x = PL(0.00919840162386765, W_inv, args, tol=1e-10, max_iter=10000)

Optimization terminated successfully.
         Current function value: 0.504464
         Iterations: 345
         Function evaluations: 1025


In [43]:
α[1:]

array([-1.68      ,  3.68      , -0.39      , -0.87      , -1.36      ,
        3.36      ,  0.73      , -6.54      , -0.28      ,  2.28      ,
       -3.77      , -7.08      ,  0.14115504,  0.29022421])

In [52]:
f[:,1] + f[:,0]

array([ 0.00000000e+00,  9.71445147e-17,  0.00000000e+00,  0.00000000e+00,
        9.71445147e-17,  1.00613962e-16,  0.00000000e+00,  0.00000000e+00,
        9.71445147e-17,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  8.32667268e-17, -1.04083409e-16,
        1.11022302e-16,  0.00000000e+00,  9.71445147e-17,  0.00000000e+00,
        0.00000000e+00, -9.71445147e-17,  0.00000000e+00,  1.94289029e-16,
        9.71445147e-17, -1.04083409e-16, -9.71445147e-17,  0.00000000e+00,
        3.98986399e-17, -1.00613962e-16,  0.00000000e+00,  1.00613962e-16,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        9.71445147e-17, -9.71445147e-17,  0.00000000e+00,  9.71445147e-17,
        0.00000000e+00,  1.00613962e-16,  1.09287579e-16, -1.00613962e-16,
        9.71445147e-17,  0.00000000e+00,  9.71445147e-17,  2.01227923e-16,
       -1.04083409e-16,  0.00000000e+00, -9.71445147e-17,  0.00000000e+00,
        0.00000000e+00,  

In [50]:
f[:,0]

array([-0.07891775, -0.08327423, -0.11836349, -0.03308017, -0.09887741,
       -0.01685118, -0.05383783, -0.06808392,  0.01689001,  0.03859749,
       -0.03876882,  0.03737163, -0.06527585,  0.11328411,  0.06301702,
       -0.06015178, -0.07895613, -0.10194566, -0.09898884, -0.01928556,
       -0.05018335,  0.03290199, -0.05093985,  0.08023504, -0.03126124,
        0.05717883, -0.08074165, -0.11562218,  0.00409018, -0.03092411,
       -0.06541079,  0.02851522,  0.27705188, -0.02809355, -0.11018416,
       -0.0529409 , -0.04048583, -0.02947586, -0.03400492, -0.05089908,
       -0.02728303, -0.02906991, -0.00581481, -0.02510561,  0.03325063,
       -0.06664818, -0.0345182 ,  0.03154912,  0.05265212,  0.11508254,
       -0.05133489, -0.11591167, -0.01672623, -0.06368954, -0.00128439,
        0.07655299, -0.10682545, -0.02518353, -0.0176502 ,  0.04525715,
        0.06013468,  0.05881944,  0.01648393,  0.0424661 ,  0.27310845,
       -0.14008835, -0.07459866, -0.09428814,  0.00991487,  0.01